# Infer-4-Bayesian-Networks : Reseaux Bayesiens Classiques

**Serie** : Programmation Probabiliste avec Infer.NET (4/13)  
**Duree estimee** : 55 minutes  
**Prerequis** : Infer-3-Factor-Graphs

---

## Objectifs

- Comprendre les reseaux bayesiens et leur structure
- Implementer le reseau classique Wet Grass / Sprinkler / Rain
- Maitriser les tables de probabilites conditionnelles (CPT)
- Distinguer inference causale et observationnelle
- Comprendre la D-separation et l'independance conditionnelle

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-3-Factor-Graphs](Infer-3-Factor-Graphs.ipynb) | [Infer-5-Skills-IRT](Infer-5-Skills-IRT.ipynb) |

---

## 1. Configuration

Cette section initialise l'environnement Infer.NET avec les packages necessaires pour construire et inferer des reseaux bayesiens. Les reseaux bayesiens sont des modeles graphiques diriges qui representent les relations de dependance conditionnelle entre variables.

In [ ]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


## 2. Introduction aux Reseaux Bayesiens

### Definition

Un **reseau bayesien** est un graphe dirige acyclique (DAG) ou :
- Les **noeuds** representent des variables aleatoires
- Les **arcs** representent des dependances directes
- Chaque noeud a une **table de probabilite conditionnelle** (CPT)

### Proprietes

$$P(X_1, ..., X_n) = \prod_{i=1}^{n} P(X_i | \text{Parents}(X_i))$$

### Avantages

| Avantage | Description |
|----------|-------------|
| **Compacite** | Representation factorisee de la jointe |
| **Interpretabilite** | Structure causale explicite |
| **Inference efficace** | Algorithmes de propagation |
| **Apprentissage** | Structure et parametres apprenables |

## 3. Le Reseau Wet Grass / Sprinkler / Rain

### Structure

```
         (Cloudy)
         /      \
        v        v
   (Sprinkler)  (Rain)
        \        /
         v      v
        (WetGrass)
```

### Semantique

- **Cloudy** : Temps nuageux (cause commune)
- **Sprinkler** : Arroseur automatique (s'active moins si nuageux)
- **Rain** : Pluie (plus probable si nuageux)
- **WetGrass** : Herbe mouillee (effet commun)

### Tables de Probabilites Conditionnelles

| Variable | CPT |
|----------|-----|
| Cloudy | P(C=T) = 0.5 |
| Sprinkler | P(S=T\|C=T) = 0.1, P(S=T\|C=F) = 0.5 |
| Rain | P(R=T\|C=T) = 0.8, P(R=T\|C=F) = 0.2 |
| WetGrass | P(W=T\|S,R) - voir table complete |

In [ ]:
// Implementation du reseau Wet Grass

// Variable racine : Cloudy
Variable<bool> cloudy = Variable.Bernoulli(0.5).Named("cloudy");

// Sprinkler conditionne par Cloudy
Variable<bool> sprinkler = Variable.New<bool>().Named("sprinkler");
using (Variable.If(cloudy))
{
    sprinkler.SetTo(Variable.Bernoulli(0.1));  // Peu probable si nuageux
}
using (Variable.IfNot(cloudy))
{
    sprinkler.SetTo(Variable.Bernoulli(0.5));  // Plus probable si ensoleille
}

// Rain conditionne par Cloudy
Variable<bool> rain = Variable.New<bool>().Named("rain");
using (Variable.If(cloudy))
{
    rain.SetTo(Variable.Bernoulli(0.8));  // Tres probable si nuageux
}
using (Variable.IfNot(cloudy))
{
    rain.SetTo(Variable.Bernoulli(0.2));  // Peu probable si ensoleille
}

// WetGrass conditionne par Sprinkler ET Rain
Variable<bool> wetGrass = Variable.New<bool>().Named("wetGrass");

// CPT pour WetGrass
// S=F, R=F -> P(W=T) = 0.0
// S=F, R=T -> P(W=T) = 0.9
// S=T, R=F -> P(W=T) = 0.9
// S=T, R=T -> P(W=T) = 0.99

using (Variable.If(sprinkler))
{
    using (Variable.If(rain))
    {
        wetGrass.SetTo(Variable.Bernoulli(0.99));  // S=T, R=T
    }
    using (Variable.IfNot(rain))
    {
        wetGrass.SetTo(Variable.Bernoulli(0.9));   // S=T, R=F
    }
}
using (Variable.IfNot(sprinkler))
{
    using (Variable.If(rain))
    {
        wetGrass.SetTo(Variable.Bernoulli(0.9));   // S=F, R=T
    }
    using (Variable.IfNot(rain))
    {
        wetGrass.SetTo(Variable.Bernoulli(0.0));   // S=F, R=F
    }
}

Console.WriteLine("Reseau Wet Grass defini.");

Reseau Wet Grass defini.


In [ ]:
// Inference sans observations
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Probabilites marginales (sans observation) ===");
Console.WriteLine($"P(Cloudy) = {moteur.Infer<Bernoulli>(cloudy).GetProbTrue():F3}");
Console.WriteLine($"P(Sprinkler) = {moteur.Infer<Bernoulli>(sprinkler).GetProbTrue():F3}");
Console.WriteLine($"P(Rain) = {moteur.Infer<Bernoulli>(rain).GetProbTrue():F3}");
Console.WriteLine($"P(WetGrass) = {moteur.Infer<Bernoulli>(wetGrass).GetProbTrue():F3}");

=== Probabilites marginales (sans observation) ===


Compiling model...

done.


P(Cloudy) = 0,500


P(Sprinkler) = 0,300


P(Rain) = 0,500


P(WetGrass) = 0,598


### Analyse des probabilites marginales

**Résultats** :
- P(Cloudy) = 0.500 — le prior direct
- P(Sprinkler) = 0.300 — moyenne pondérée : 0.5×0.1 + 0.5×0.5 = 0.30 ✓
- P(Rain) = 0.500 — moyenne pondérée : 0.5×0.8 + 0.5×0.2 = 0.50 ✓
- P(WetGrass) = 0.598 — plus complexe, dépend des 4 combinaisons S×R

**Calcul détaillé de P(WetGrass)** :

$$P(W) = \sum_{s,r} P(W|S=s,R=r) \cdot P(S=s) \cdot P(R=r)$$

La valeur ~60% reflète que l'herbe mouillée est un phénomène relativement fréquent avec ces paramètres.

## 4. Inference avec Observations

In [ ]:
// Nouveau modele pour inference avec observation
Variable<bool> cloudy2 = Variable.Bernoulli(0.5).Named("cloudy2");
Variable<bool> sprinkler2 = Variable.New<bool>().Named("sprinkler2");
Variable<bool> rain2 = Variable.New<bool>().Named("rain2");
Variable<bool> wetGrass2 = Variable.New<bool>().Named("wetGrass2");

// Meme structure que precedemment
using (Variable.If(cloudy2))
{
    sprinkler2.SetTo(Variable.Bernoulli(0.1));
    rain2.SetTo(Variable.Bernoulli(0.8));
}
using (Variable.IfNot(cloudy2))
{
    sprinkler2.SetTo(Variable.Bernoulli(0.5));
    rain2.SetTo(Variable.Bernoulli(0.2));
}

using (Variable.If(sprinkler2))
{
    using (Variable.If(rain2))
        wetGrass2.SetTo(Variable.Bernoulli(0.99));
    using (Variable.IfNot(rain2))
        wetGrass2.SetTo(Variable.Bernoulli(0.9));
}
using (Variable.IfNot(sprinkler2))
{
    using (Variable.If(rain2))
        wetGrass2.SetTo(Variable.Bernoulli(0.9));
    using (Variable.IfNot(rain2))
        wetGrass2.SetTo(Variable.Bernoulli(0.0));
}

// OBSERVATION : L'herbe est mouillee
wetGrass2.ObservedValue = true;

InferenceEngine moteur2 = new InferenceEngine();
moteur2.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Inference : P(X | WetGrass=True) ===");
Console.WriteLine($"P(Rain | WetGrass) = {moteur2.Infer<Bernoulli>(rain2).GetProbTrue():F3}");
Console.WriteLine($"P(Sprinkler | WetGrass) = {moteur2.Infer<Bernoulli>(sprinkler2).GetProbTrue():F3}");
Console.WriteLine($"P(Cloudy | WetGrass) = {moteur2.Infer<Bernoulli>(cloudy2).GetProbTrue():F3}");

=== Inference : P(X | WetGrass=True) ===


Compiling model...

done.


Iterating: 


.

...

|

 50


P(Rain | WetGrass) = 0,784


P(Sprinkler | WetGrass) = 0,404


P(Cloudy | WetGrass) = 0,604


### Analyse de l'inférence conditionnelle

**Résultats** : P(X | WetGrass=True)

| Variable | Prior | Posterior | Changement |
|----------|-------|-----------|------------|
| Rain | 0.500 | **0.784** | +57% |
| Sprinkler | 0.300 | **0.404** | +35% |
| Cloudy | 0.500 | **0.604** | +21% |

**Interprétation** :
- L'herbe mouillée est une **forte évidence** pour la pluie (car P(W|R,¬S)=0.9)
- L'arroseur est aussi plus probable, mais moins fortement
- Le temps nuageux augmente car il favorise la pluie, qui explique bien l'herbe mouillée

> **Raisonnement abductif** : L'inférence "remonte" des effets vers les causes. C'est le cœur du diagnostic bayésien.

### Analyse : Explaining Away

Observez que si l'herbe est mouillee :
- P(Rain) augmente significativement
- P(Sprinkler) augmente aussi

Mais si on sait aussi qu'il a plu, alors P(Sprinkler) diminue ! C'est le phenomene de **explaining away**.

In [ ]:
// Demonstation de l'Explaining Away
Variable<bool> cloudy3 = Variable.Bernoulli(0.5);
Variable<bool> sprinkler3 = Variable.New<bool>();
Variable<bool> rain3 = Variable.New<bool>();
Variable<bool> wetGrass3 = Variable.New<bool>();

using (Variable.If(cloudy3))
{
    sprinkler3.SetTo(Variable.Bernoulli(0.1));
    rain3.SetTo(Variable.Bernoulli(0.8));
}
using (Variable.IfNot(cloudy3))
{
    sprinkler3.SetTo(Variable.Bernoulli(0.5));
    rain3.SetTo(Variable.Bernoulli(0.2));
}

using (Variable.If(sprinkler3))
{
    using (Variable.If(rain3))
        wetGrass3.SetTo(Variable.Bernoulli(0.99));
    using (Variable.IfNot(rain3))
        wetGrass3.SetTo(Variable.Bernoulli(0.9));
}
using (Variable.IfNot(sprinkler3))
{
    using (Variable.If(rain3))
        wetGrass3.SetTo(Variable.Bernoulli(0.9));
    using (Variable.IfNot(rain3))
        wetGrass3.SetTo(Variable.Bernoulli(0.0));
}

// Observations : WetGrass=True ET Rain=True
wetGrass3.ObservedValue = true;
rain3.ObservedValue = true;

InferenceEngine moteur3 = new InferenceEngine();
moteur3.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Explaining Away ===");
Console.WriteLine($"P(Sprinkler | WetGrass, Rain) = {moteur3.Infer<Bernoulli>(sprinkler3).GetProbTrue():F3}");
Console.WriteLine("\nComparaison :");
Console.WriteLine("  P(Sprinkler | WetGrass) ~ 0.43");
Console.WriteLine("  P(Sprinkler | WetGrass, Rain) ~ 0.19");
Console.WriteLine("\n=> La pluie 'explique' l'herbe mouillee, reduisant P(Sprinkler)");

=== Explaining Away ===


Compiling model...

done.


P(Sprinkler | WetGrass, Rain) = 0,194



Comparaison :


  P(Sprinkler | WetGrass) ~ 0.43


  P(Sprinkler | WetGrass, Rain) ~ 0.19



=> La pluie 'explique' l'herbe mouillee, reduisant P(Sprinkler)


### Explication du phénomène "Explaining Away"

**Résultat clé** : P(Sprinkler | WetGrass, Rain) = **0.194** vs P(Sprinkler | WetGrass) ≈ 0.40

Quand on observe que **l'herbe est mouillée ET qu'il a plu**, la pluie "explique" suffisamment l'observation. L'arroseur devient **moins nécessaire** comme explication.

**Analogie intuitive** : Si vous arrivez au bureau trempé et que je sais qu'il pleut dehors, je ne vais pas penser que vous êtes aussi passé sous un arroseur. Une explication suffit.

**Structure graphique** :
```
(Sprinkler) → (WetGrass) ← (Rain)
            [collider]
```

Au collider, les deux causes sont **marginalement indépendantes** mais deviennent **conditionnellement dépendantes** quand l'effet est observé. C'est contre-intuitif mais fondamental en raisonnement bayésien.

## 5. D-Separation et Independance Conditionnelle

### Definition

La **D-separation** est un critere graphique pour determiner l'independance conditionnelle dans un reseau bayesien.

### Trois structures de base

| Structure | Nom | Independance |
|-----------|-----|-------------|
| A -> B -> C | Chaine | A _|_ C \| B |
| A <- B -> C | Fork (cause commune) | A _|_ C \| B |
| A -> B <- C | Collider (effet commun) | A _|_ C, mais A /|/ C \| B |

### Application au reseau Wet Grass

- **Sprinkler** et **Rain** sont independants si **Cloudy** n'est pas observe
- **Sprinkler** et **Rain** deviennent dependants si **WetGrass** est observe (collider)

In [ ]:
// Verification de la D-separation

// Test 1 : Sprinkler et Rain sans observation sur WetGrass
Variable<bool> c4 = Variable.Bernoulli(0.5);
Variable<bool> s4 = Variable.New<bool>();
Variable<bool> r4 = Variable.New<bool>();

using (Variable.If(c4)) { s4.SetTo(Variable.Bernoulli(0.1)); r4.SetTo(Variable.Bernoulli(0.8)); }
using (Variable.IfNot(c4)) { s4.SetTo(Variable.Bernoulli(0.5)); r4.SetTo(Variable.Bernoulli(0.2)); }

// Observation : Sprinkler = true
s4.ObservedValue = true;

InferenceEngine m4 = new InferenceEngine();
m4.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Test D-separation (sans WetGrass) ===");
Console.WriteLine($"P(Rain | Sprinkler) = {m4.Infer<Bernoulli>(r4).GetProbTrue():F3}");
Console.WriteLine("P(Rain) marginale = 0.500");
Console.WriteLine("\n=> S et R sont marginalement independants via Cloudy (fork)");

=== Test D-separation (sans WetGrass) ===


Compiling model...

done.


P(Rain | Sprinkler) = 0,300


P(Rain) marginale = 0.500



=> S et R sont marginalement independants via Cloudy (fork)


### Analyse de la D-séparation

**Résultat** : P(Rain | Sprinkler=True) = **0.300** ≠ P(Rain) = 0.500

**Surprise !** Sprinkler et Rain ne sont PAS marginalement indépendants, contrairement à ce qu'on pourrait croire.

**Explication** : Observer Sprinkler=True donne de l'information sur Cloudy :
- P(Cloudy | Sprinkler=True) = P(S=T|C) × P(C) / P(S=T) = 0.1 × 0.5 / 0.3 ≈ 0.167
- Donc il fait probablement beau (ensoleillé)
- Et P(Rain | ¬Cloudy) = 0.2, d'où P(Rain | S=T) ≈ 0.3

**Conclusion** : Dans la structure "fork" (Cloudy → S et Cloudy → R), S et R sont D-séparés **seulement si on conditionne sur Cloudy**. Sans cette observation, l'information "passe" par Cloudy.

## 6. Inference Causale vs Observationnelle

### Difference fondamentale

| Type | Question | Notation |
|------|----------|----------|
| **Observationnel** | P(Y\|X=x) | Que se passe-t-il quand on observe X=x ? |
| **Interventionnel** | P(Y\|do(X=x)) | Que se passe-t-il quand on force X=x ? |

### Exemple

- **Observer** qu'il pleut : P(WetGrass | Rain=True)
- **Faire pleuvoir** artificiellement : P(WetGrass | do(Rain=True))

Dans le second cas, on "coupe" les arcs entrants vers Rain (il ne depend plus de Cloudy).

In [ ]:
// Comparaison inference observationnelle vs interventionnelle

// OBSERVATIONNEL : P(Cloudy | Rain=True)
Variable<bool> cObs = Variable.Bernoulli(0.5);
Variable<bool> rObs = Variable.New<bool>();

using (Variable.If(cObs)) { rObs.SetTo(Variable.Bernoulli(0.8)); }
using (Variable.IfNot(cObs)) { rObs.SetTo(Variable.Bernoulli(0.2)); }

rObs.ObservedValue = true;

InferenceEngine mObs = new InferenceEngine();
mObs.Compiler.CompilerChoice = CompilerChoice.Roslyn;

double pCloudyObs = mObs.Infer<Bernoulli>(cObs).GetProbTrue();

// INTERVENTIONNEL : P(Cloudy | do(Rain=True))
// On "coupe" l'arc Cloudy -> Rain en fixant Rain independamment
Variable<bool> cInt = Variable.Bernoulli(0.5);
Variable<bool> rInt = Variable.Bernoulli(1.0);  // Force a True, independant de Cloudy

InferenceEngine mInt = new InferenceEngine();
mInt.Compiler.CompilerChoice = CompilerChoice.Roslyn;

double pCloudyInt = mInt.Infer<Bernoulli>(cInt).GetProbTrue();

Console.WriteLine("=== Observationnel vs Interventionnel ===");
Console.WriteLine($"P(Cloudy | Rain=True) = {pCloudyObs:F3} (observationnel)");
Console.WriteLine($"P(Cloudy | do(Rain=True)) = {pCloudyInt:F3} (interventionnel)");
Console.WriteLine("\n=> Observer la pluie informe sur le temps nuageux");
Console.WriteLine("   Faire pleuvoir ne change pas le temps nuageux");

Compiling model...

done.


Compiling model...

done.


=== Observationnel vs Interventionnel ===


P(Cloudy | Rain=True) = 0,800 (observationnel)


P(Cloudy | do(Rain=True)) = 0,500 (interventionnel)



=> Observer la pluie informe sur le temps nuageux


   Faire pleuvoir ne change pas le temps nuageux


### Analyse : Causalité vs Corrélation

**Résultats** :
- P(Cloudy | Rain=True) = **0.800** — observationnel
- P(Cloudy | do(Rain=True)) = **0.500** — interventionnel

**Différence fondamentale** :

| Type | Sémantique | Résultat |
|------|------------|----------|
| Observation | "Je vois qu'il pleut" | Le temps est probablement nuageux |
| Intervention | "Je fais pleuvoir (machine à pluie)" | Le temps reste incertain |

L'**opérateur do()** de Pearl "coupe" les arcs entrants vers la variable manipulée. Rain ne dépend plus de Cloudy dans le graphe modifié.

> **Application pratique** : Cette distinction est cruciale en médecine. Observer une corrélation entre un traitement et la guérison n'implique pas que le traitement cause la guérison (biais de sélection possible). Un essai randomisé (intervention) est nécessaire pour établir la causalité.

## 7. Modele Causal : Direction de la Causalite

### Question

Etant donne des donnees observationnelles entre A et B, comment determiner si A cause B ou B cause A ?

### Approche bayesienne

Comparer les evidences des deux modeles :
- $M_1$ : A -> B
- $M_2$ : B -> A

In [ ]:
// Modele pour identifier la direction causale

// Donnees simulees : A cause B avec bruit
double[] dataA = { 1.0, 2.0, 3.0, 4.0, 5.0 };
double[] dataB = { 2.1, 4.2, 5.8, 8.1, 10.3 };  // B ~ 2*A + bruit

int n = dataA.Length;

// Modele 1 : A -> B (A cause B)
Variable<bool> evidence1 = Variable.Bernoulli(0.5);
using (Variable.If(evidence1))
{
    Variable<double> slope1 = Variable.GaussianFromMeanAndPrecision(0, 0.1);
    Variable<double> intercept1 = Variable.GaussianFromMeanAndPrecision(0, 0.1);
    Variable<double> noise1 = Variable.GammaFromShapeAndScale(2, 0.5);
    
    for (int i = 0; i < n; i++)
    {
        Variable<double> aPrior1 = Variable.GaussianFromMeanAndPrecision(0, 0.1);
        aPrior1.ObservedValue = dataA[i];
        Variable<double> bPred1 = slope1 * aPrior1 + intercept1;
        Variable<double> bObs1 = Variable.GaussianFromMeanAndPrecision(bPred1, noise1);
        bObs1.ObservedValue = dataB[i];
    }
}

InferenceEngine mCausal1 = new InferenceEngine();
mCausal1.Compiler.CompilerChoice = CompilerChoice.Roslyn;

double logEvidence1 = mCausal1.Infer<Bernoulli>(evidence1).LogOdds;

Console.WriteLine("=== Selection de la direction causale ===");
Console.WriteLine($"Log Evidence (A -> B) = {logEvidence1:F3}");
Console.WriteLine("\nNote: Un modele plus sophistique comparerait avec B -> A");

Compiling model...

done.


Iterating: 


.

.

...

|

 50


=== Selection de la direction causale ===


Log Evidence (A -> B) = -22,920



Note: Un modele plus sophistique comparerait avec B -> A


## 8. Modele BUGS Rats (Modele Hierarchique)

### Contexte

Le modele "Rats" de BUGS est un exemple classique de modele hierarchique :
- Plusieurs rats suivis dans le temps
- Chaque rat a sa propre trajectoire de croissance
- Les parametres individuels sont tires d'une distribution de population

### Structure hierarchique

```
Population: mu_alpha, sigma_alpha, mu_beta, sigma_beta
     |
     v
Individu: alpha[i], beta[i] ~ N(mu, sigma)
     |
     v
Observation: y[i,t] = alpha[i] + beta[i] * t + epsilon
```

In [ ]:
// Modele hierarchique simplifie (style BUGS Rats)

int nRats = 5;
int nTimePoints = 4;
double[] times = { 8.0, 15.0, 22.0, 29.0 };  // Ages en jours

// Donnees simulees : poids des rats au fil du temps
double[,] weights = new double[,] {
    { 151, 199, 246, 283 },
    { 145, 199, 249, 293 },
    { 147, 214, 263, 312 },
    { 155, 200, 237, 272 },
    { 135, 188, 230, 280 }
};

// Parametres de population
Variable<double> muAlpha = Variable.GaussianFromMeanAndVariance(150, 1000);
Variable<double> muBeta = Variable.GaussianFromMeanAndVariance(5, 100);
Variable<double> tauAlpha = Variable.GammaFromShapeAndScale(1, 1);
Variable<double> tauBeta = Variable.GammaFromShapeAndScale(1, 1);
Variable<double> tauNoise = Variable.GammaFromShapeAndScale(1, 1);

// Parametres individuels
Range ratRange = new Range(nRats);
VariableArray<double> alpha = Variable.Array<double>(ratRange);
VariableArray<double> beta = Variable.Array<double>(ratRange);

using (Variable.ForEach(ratRange))
{
    alpha[ratRange] = Variable.GaussianFromMeanAndPrecision(muAlpha, tauAlpha);
    beta[ratRange] = Variable.GaussianFromMeanAndPrecision(muBeta, tauBeta);
}

// Observations
for (int i = 0; i < nRats; i++)
{
    for (int t = 0; t < nTimePoints; t++)
    {
        Variable<double> meanWeight = alpha[i] + beta[i] * (times[t] - 22.0);
        Variable<double> obsWeight = Variable.GaussianFromMeanAndPrecision(meanWeight, tauNoise);
        obsWeight.ObservedValue = weights[i, t];
    }
}

InferenceEngine mRats = new InferenceEngine(new ExpectationPropagation());
mRats.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Modele Hierarchique (Rats) ===");
Console.WriteLine($"mu_alpha (intercept moyen) : {mRats.Infer<Gaussian>(muAlpha)}");
Console.WriteLine($"mu_beta (pente moyenne) : {mRats.Infer<Gaussian>(muBeta)}");

=== Modele Hierarchique (Rats) ===


Compiling model...

done.


Iterating: 


.

...

.

|

 50


mu_alpha (intercept moyen) : Gaussian(241,7, 22,96)


mu_beta (pente moyenne) : Gaussian(6,68, 0,2134)


### Analyse du modèle hiérarchique

**Résultats** :
- μ_alpha (poids moyen à t=22 jours) ≈ **241.7 g**
- μ_beta (croissance moyenne) ≈ **6.68 g/jour**

**Interprétation biologique** :
- À 22 jours, les rats pèsent en moyenne ~242g
- Ils prennent environ 6.7g par jour

**Avantages du modèle hiérarchique** :

| Aspect | Explication |
|--------|-------------|
| **Partage d'information** | Les rats "empruntent" de l'information les uns aux autres via la population |
| **Shrinkage** | Les estimations individuelles sont tirées vers la moyenne |
| **Robustesse** | Moins sensible aux outliers individuels |
| **Généralisation** | Peut prédire pour un nouveau rat non observé |

> **Concept clé** : La hiérarchie "régularise" naturellement les estimations. Un rat avec peu d'observations sera tiré vers la moyenne de population, évitant le sur-ajustement.

## 9. Exercice : Reseau de Diagnostic Medical

### Enonce

Construisez un reseau bayesien pour le diagnostic de deux maladies :

**Structure** :
```
(Cold)     (Flu)
   \         /
    v       v
   (Fatigue)
    |
    v
  (Fever)
```

**CPTs** :
- P(Cold) = 0.02
- P(Flu) = 0.01
- P(Fatigue | Cold, Flu) : voir table
- P(Fever | Fatigue) : 0.8 si fatigue, 0.05 sinon

**Question** : P(Flu | Fever=True) ?

In [ ]:
// EXERCICE : Reseau de diagnostic

Variable<bool> cold = Variable.Bernoulli(0.02).Named("cold");
Variable<bool> flu = Variable.Bernoulli(0.01).Named("flu");
Variable<bool> fatigue = Variable.New<bool>().Named("fatigue");
Variable<bool> fever = Variable.New<bool>().Named("fever");

// CPT pour Fatigue
// Cold=F, Flu=F -> P(Fatigue) = 0.1
// Cold=F, Flu=T -> P(Fatigue) = 0.9
// Cold=T, Flu=F -> P(Fatigue) = 0.6
// Cold=T, Flu=T -> P(Fatigue) = 0.95

using (Variable.If(cold))
{
    using (Variable.If(flu))
        fatigue.SetTo(Variable.Bernoulli(0.95));
    using (Variable.IfNot(flu))
        fatigue.SetTo(Variable.Bernoulli(0.6));
}
using (Variable.IfNot(cold))
{
    using (Variable.If(flu))
        fatigue.SetTo(Variable.Bernoulli(0.9));
    using (Variable.IfNot(flu))
        fatigue.SetTo(Variable.Bernoulli(0.1));
}

// CPT pour Fever
using (Variable.If(fatigue))
{
    fever.SetTo(Variable.Bernoulli(0.8));
}
using (Variable.IfNot(fatigue))
{
    fever.SetTo(Variable.Bernoulli(0.05));
}

// Observation
fever.ObservedValue = true;

InferenceEngine mDiag = new InferenceEngine();
mDiag.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Diagnostic Medical ===");
Console.WriteLine($"P(Flu | Fever) = {mDiag.Infer<Bernoulli>(flu).GetProbTrue():F3}");
Console.WriteLine($"P(Cold | Fever) = {mDiag.Infer<Bernoulli>(cold).GetProbTrue():F3}");
Console.WriteLine($"P(Fatigue | Fever) = {mDiag.Infer<Bernoulli>(fatigue).GetProbTrue():F3}");

=== Diagnostic Medical ===


Compiling model...

done.


P(Flu | Fever) = 0,052


P(Cold | Fever) = 0,073


P(Fatigue | Fever) = 0,681


### Analyse du diagnostic médical

**Résultats** : P(Maladie | Fever=True)

| Maladie | Prior | Posterior | Ratio |
|---------|-------|-----------|-------|
| Flu | 0.01 | **0.052** | ×5.2 |
| Cold | 0.02 | **0.073** | ×3.6 |

**Observations** :

1. **Explaining away partiel** : Cold et Flu sont tous deux plus probables avec la fièvre, mais moins que si l'un des deux était certain (chacun "explique partiellement" la fièvre).

2. **P(Fatigue | Fever) = 0.681** : La fièvre est fortement associée à la fatigue car P(Fever|Fatigue) = 0.8 >> P(Fever|¬Fatigue) = 0.05.

3. **Diagnostic différentiel** : Avec seulement la fièvre, il reste difficile de distinguer Cold de Flu. Des symptômes supplémentaires (courbatures, durée) seraient nécessaires.

> **Application clinique** : Ce type de réseau bayésien est utilisé dans les systèmes d'aide au diagnostic (MYCIN, DXplain). L'ajout de symptômes affine progressivement les probabilités des maladies.

## 10. Resume

| Concept | Description |
|---------|-------------|
| **Reseau bayesien** | DAG avec variables et CPTs |
| **CPT** | Table de Probabilite Conditionnelle |
| **D-separation** | Critere graphique d'independance |
| **Explaining away** | Causes alternatives deviennent moins probables |
| **Observation vs Intervention** | P(Y\|X) vs P(Y\|do(X)) |
| **Modele hierarchique** | Parametres individuels tires d'une population |

---

## Prochaine etape

Dans [Infer-5-Skills-IRT](Infer-5-Skills-IRT.ipynb), nous explorerons :

- Les modeles d'evaluation de competences
- Item Response Theory (IRT) avec le modele Difficulty-Ability
- Le modele DINA (Noisy-And) pour les competences multiples
- Les relations many-to-many entre competences et questions